### This example is taken from https://github.com/fchollet/deep-learning-with-python-notebooks

In [ ]:
%pylab inline

In [ ]:
import keras
keras.__version__

# Problem statement

The problem we are trying to solve here is to classify grayscale images of handwritten digits (28 pixels by 28 pixels), into their 10 
categories (0 to 9). The dataset we will use is the MNIST dataset, a classic dataset in the machine learning community, which has been 
around for almost as long as the field itself and has been very intensively studied. It's a set of 60,000 training images, plus 10,000 test 
images, assembled by the National Institute of Standards and Technology (the NIST in MNIST) in the 1980s. You can think of "solving" MNIST 
as the "Hello World" of deep learning -- it's what you do to verify that your algorithms are working as expected. As you become a machine 
learning practitioner, you will see MNIST come up over and over again, in scientific papers, blog posts, and so on.

## Keras based solution

The MNIST dataset comes pre-loaded in Keras, in the form of a set of four Numpy arrays:

In [ ]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

`train_images` and `train_labels` form the "training set", the data that the model will learn from. The model will then be tested on the 
"test set", `test_images` and `test_labels`. Our images are encoded as Numpy arrays, and the labels are simply an array of digits, ranging 
from 0 to 9. There is a one-to-one correspondence between the images and the labels.

Let's have a look at the training data:

In [ ]:
train_images.shape

In [ ]:
len(train_labels)

In [ ]:
train_labels

In [ ]:
figure(figsize=(15,15))

for i in range(36):
    subplot(6,6,i+1)
    imshow(train_images[i], cmap='Greys')
    title(train_labels[i])
    #axis('off')
    tick_params(labelbottom=False)
    tick_params(labelleft=False)

Let's have a look at the test data:

In [ ]:
test_images.shape

In [ ]:
len(test_labels)

In [ ]:
test_labels

Our workflow will be as follow: first we will present our neural network with the training data, `train_images` and `train_labels`. The 
network will then learn to associate images and labels. Finally, we will ask the network to produce predictions for `test_images`, and we 
will verify if these predictions match the labels from `test_labels`.

Let's build our network -- again, remember that you aren't supposed to understand everything about this example just yet.

### Network architecture

In [ ]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

In [ ]:
network.summary()


The core building block of neural networks is the "layer", a data-processing module which you can conceive as a "filter" for data. Some 
data comes in, and comes out in a more useful form. Precisely, layers extract _representations_ out of the data fed into them -- hopefully 
representations that are more meaningful for the problem at hand. Most of deep learning really consists of chaining together simple layers 
which will implement a form of progressive "data distillation". A deep learning model is like a sieve for data processing, made of a 
succession of increasingly refined data filters -- the "layers".

Here our network consists of a sequence of two `Dense` layers, which are densely-connected (also called "fully-connected") neural layers. 
The second (and last) layer is a 10-way "softmax" layer, which means it will return an array of 10 probability scores (summing to 1). Each 
score will be the probability that the current digit image belongs to one of our 10 digit classes.

To make our network ready for training, we need to pick three more things, as part of "compilation" step:

* A loss function: the is how the network will be able to measure how good a job it is doing on its training data, and thus how it will be 
able to steer itself in the right direction.
* An optimizer: this is the mechanism through which the network will update itself based on the data it sees and its loss function.
* Metrics to monitor during training and testing. Here we will only care about accuracy (the fraction of the images that were correctly 
classified).

The exact purpose of the loss function and the optimizer will be made clear throughout the next two chapters.

In [ ]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])


Before training, we will preprocess our data by reshaping it into the shape that the network expects, and scaling it so that all values are in 
the `[0, 1]` interval. Previously, our training images for instance were stored in an array of shape `(60000, 28, 28)` of type `uint8` with 
values in the `[0, 255]` interval. We transform it into a `float32` array of shape `(60000, 28 * 28)` with values between 0 and 1.

In [ ]:
# Reshape of 2D images to one dim vector. Scale change: values to the range [0:1] 

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

We also need to categorically encode the labels, a step which we explain in chapter 3:

In [ ]:
from keras.utils import to_categorical

print ("Labels before one-hot encoding: ", test_labels[0])

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print ("Labels after one-hot encoding: ", test_labels[0])

### Model fit: Training step

We are now ready to train our network, which in Keras is done via a call to the `fit` method of the network: 
we "fit" the model to its training data.

In [ ]:
network.fit(train_images, train_labels, epochs=10, batch_size=128)

Two quantities are being displayed during training: the "loss" of the network over the training data, and the accuracy of the network over 
the training data.

We quickly reach an accuracy of 0.989 (i.e. 98.9%) on the training data. Now let's check that our model performs well on the test set too:

In [ ]:
history = network.history

epochs = history.epoch
acc = history.history['acc']
loss = history.history['loss']

In [ ]:
fig = figure(figsize(8,5))
ax1 = fig.subplots()

ax1.plot(epochs, acc, 'b-')
ax1.set_xlabel('epoch', fontsize=14)
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('Accuracy', color='b', fontsize=14)
ax1.tick_params('y', colors='b')

ax2 = ax1.twinx()
ax2.plot(epochs, loss, 'r--')
ax2.set_ylabel('Loss', color='r', fontsize=14)
ax2.tick_params('y', colors='r')

fig.tight_layout()
plt.show()

### Model evaluation: Testing phase

In [ ]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

In [ ]:
print('test_acc:', test_acc)

Our test set accuracy turns out to be about 97-98% -- that's quite a bit lower than the training set accuracy. 
This gap between training accuracy and test accuracy is an example of "overfitting", 
the fact that machine learning models tend to perform worse on new data than on their training data. 
Overfitting will be a central topic in chapter 3.

This concludes our very first example -- you just saw how we could build and a train a neural network to classify handwritten digits, in 
less than 20 lines of Python code. In the next chapter, we will go in detail over every moving piece we just previewed, and clarify what is really 
going on behind the scenes. You will learn about "tensors", the data-storing objects going into the network, about tensor operations, which 
layers are made of, and about gradient descent, which allows our network to learn from its training examples.

### Whole code  
Just to see that we used a bunch of lines of code to implement an end to end Deep Learning solution

In [ ]:
from keras.datasets import mnist
from keras import models
from keras import layers
from keras.utils import to_categorical

# Load data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

 # Preparing data
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(train_images, train_labels, epochs=5, batch_size=128)

test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_acc:', test_acc)

## Validation set and overfitting

Now we split the training set in two parts by using the validation_split parameter of the fit method.
validation_split=0.1 means that the 10% of the training set is used as validation set

#### Training using validation set too

In [ ]:
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(train_images, train_labels, epochs=10, batch_size=128, validation_split=0.1)

In [ ]:
history = network.history

print(history.history.keys())
epochs = history.epoch
acc = history.history['acc']
loss = history.history['loss']
val_acc = history.history['val_acc']
val_loss = history.history['val_loss']

fig = figure(figsize(16,5))
ax1 = subplot(1,2,1)

ax1.plot(epochs, acc, 'b-', label='acc')
ax1.plot(epochs, val_acc, 'r--', label='loss_acc')
ax1.set_xlabel('epoch', fontsize=14)
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('Accuracy', color='b', fontsize=14)
ax1.tick_params('y', colors='b')
ax1.legend()
ax1.set_ylim(0.90, 1.0)

#ax2 = ax1.twinx()
ax2 = subplot(1,2,2)
ax2.plot(epochs, loss, 'b-', label='loss')
ax2.plot(epochs, val_loss, 'r--', label='val_loss')
ax2.set_ylabel('Loss', color='r', fontsize=14)
ax2.tick_params('y', colors='r')
ax2.legend()

fig.tight_layout()
plt.show()

### Trying to reduce overfitting: Reducing network size

In [ ]:
network = models.Sequential()
network.add(layers.Dense(64, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(train_images, train_labels, epochs=10, batch_size=128, validation_split=0.1)



In [ ]:
history = network.history

epochs = history.epoch
acc = history.history['acc']
loss = history.history['loss']
val_acc = history.history['val_acc']
val_loss = history.history['val_loss']


fig = figure(figsize(16,5))
ax1 = subplot(1,2,1)

ax1.plot(epochs, acc, 'b-', label='acc')
ax1.plot(epochs, val_acc, 'r--', label='loss_acc')
ax1.set_xlabel('epoch', fontsize=14)
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('Accuracy', color='b', fontsize=14)
ax1.tick_params('y', colors='b')
ax1.legend()
ax1.set_ylim(0.90, 1.0)

#ax2 = ax1.twinx()
ax2 = subplot(1,2,2)
ax2.plot(epochs, loss, 'b-', label='loss')
ax2.plot(epochs, val_loss, 'r--', label='val_loss')
ax2.set_ylabel('Loss', color='r', fontsize=14)
ax2.tick_params('y', colors='r')
ax2.legend()

fig.tight_layout()
plt.show()

test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_acc:', test_acc)

### Trying to reduce overfitting: Regularization

In [ ]:
from keras import regularizers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,), kernel_regularizer=regularizers.l2(0.001)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(train_images, train_labels, epochs=20, batch_size=128, validation_split=0.1)

In [ ]:
history = network.history

epochs = history.epoch
acc = history.history['acc']
loss = history.history['loss']
val_acc = history.history['val_acc']
val_loss = history.history['val_loss']


fig = figure(figsize(16,5))
ax1 = subplot(1,2,1)

ax1.plot(epochs, acc, 'b-', label='acc')
ax1.plot(epochs, val_acc, 'r--', label='loss_acc')
ax1.set_xlabel('epoch', fontsize=14)
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('Accuracy', color='b', fontsize=14)
ax1.tick_params('y', colors='b')
ax1.legend()
ax1.set_ylim(0.90, 1.0)

#ax2 = ax1.twinx()
ax2 = subplot(1,2,2)
ax2.plot(epochs, loss, 'b-', label='loss')
ax2.plot(epochs, val_loss, 'r--', label='val_loss')
ax2.set_ylabel('Loss', color='r', fontsize=14)
ax2.tick_params('y', colors='r')
ax2.legend()

fig.tight_layout()
plt.show()

test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_acc:', test_acc)